In [36]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import requests
import json
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/datasets/riyajolly/josh-data/FT Data - data.csv


In [37]:
df=pd.read_csv("/kaggle/input/datasets/riyajolly/josh-data/FT Data - data.csv")

In [38]:
df.head()

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...


In [39]:
!pip install -q datasets transformers accelerate torchaudio jiwer librosa soundfile

In [40]:
#Fixing broken urls
df["trans_url_fixed"]=df["transcription_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)
df["audio_url_fixed"]=df["rec_url_gcp"].str.replace("joshtalks-data-collection/hq_data/hi/", "upload_goai/", regex=False)

In [41]:
df

,user_id,recording_id,language,duration,rec_url_gcp,transcription_url_gcp,metadata_url_gcp,trans_url_fixed,audio_url_fixed
0,245746,825780,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,291038,825727,hi,443,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,246004,988596,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,93626,990175,hi,475,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,286851,526266,hi,522,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...
...,...,...,...,...,...,...,...,...,...
99,278010,753435,hi,589,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/887...,https://storage.googleapis.com/upload_goai/887...
100,413240,1021370,hi,1194,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
101,11057,1020918,hi,1194,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
102,93299,840793,hi,1146,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/joshtalks-data-...,https://storage.googleapis.com/upload_goai/983...,https://storage.googleapis.com/upload_goai/983...


In [42]:
#dropping the columns that are not required
df.drop(["user_id",	"recording_id",	"language",	"duration", "rec_url_gcp","transcription_url_gcp", "metadata_url_gcp"],axis=1, inplace=True)

In [43]:
df

,trans_url_fixed,audio_url_fixed
0,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
1,https://storage.googleapis.com/upload_goai/967...,https://storage.googleapis.com/upload_goai/967...
2,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
3,https://storage.googleapis.com/upload_goai/114...,https://storage.googleapis.com/upload_goai/114...
4,https://storage.googleapis.com/upload_goai/639...,https://storage.googleapis.com/upload_goai/639...
...,...,...
99,https://storage.googleapis.com/upload_goai/887...,https://storage.googleapis.com/upload_goai/887...
100,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
101,https://storage.googleapis.com/upload_goai/118...,https://storage.googleapis.com/upload_goai/118...
102,https://storage.googleapis.com/upload_goai/983...,https://storage.googleapis.com/upload_goai/983...


In [44]:
import os
os.makedirs("audio_raw", exist_ok=True)
os.makedirs("audio_16k", exist_ok=True)

In [45]:

import urllib.request
import librosa
import soundfile as sf
from tqdm import tqdm

processed_data=[]

for i, row in tqdm(df.iterrows(), total=len(df)):
  try:
    #download transcription
    trans_url=row["trans_url_fixed"]
    r=requests.get(trans_url, timeout=10)
    if r.status_code!=200:
      continue

    segments=r.json()
    full_text=" ".join([s["text"] for s in segments])

    #download audio
    audio_url=row["audio_url_fixed"]
    raw_path=f"audio_raw/{i}.wav"
    urllib.request.urlretrieve(audio_url, raw_path)

    #conver to 16kHz
    audio, sr=librosa.load(raw_path, sr=16000)
    processed_path=f"audio_16k/{i}.wav"
    sf.write(processed_path, audio, 16000)

    #store result
    processed_data.append({
        "audio": processed_path,
        "text":full_text
    })
  except Exception as e:
    print("Error at row: ",i,": ",e)

100%|██████████| 104/104 [15:55<00:00,  9.19s/it]


In [46]:
#converting this data into a dataframe
final_df=pd.DataFrame(processed_data)
final_df.head()

,audio,text
0,audio_16k/0.wav,अब काफी अच्छा होता है क्योंकि उनकी जनसंख्या बह...
1,audio_16k/1.wav,जी जी जी जी जी । जी जी जी हां उधर हां जी हा हा...
2,audio_16k/2.wav,लेकिन हम लोग इसे छुपछुप के लोगों के घर जाकर खे...
3,audio_16k/3.wav,जी जी जी जी जी मेरे तो जैसे बहुत सारी यादे हैं...
4,audio_16k/4.wav,हां जी पहले बात करते हैं विवाह की तो इस मुवी म...


In [47]:
!pip install datasets

In [48]:
from datasets import Dataset
dataset=Dataset.from_pandas(final_df)
dataset

Dataset({
    features: ['audio', 'text'],
    num_rows: 104
})

In [49]:
#converting audio to proper format since whisper needs audio in a specific format
from datasets import Audio
dataset=dataset.cast_column("audio", Audio(sampling_rate=16000))

### Now, the dataset is ready for training
Constructed a structured dataset by downloading audio files,resampling to 16000 Hz and converting the data into a proper dataset which will now be suitable for Whisper fine-tuning.

# Question 1: Part B

In [50]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration

model_name="openai/whisper-small"
processor=WhisperProcessor.from_pretrained(model_name)
model=WhisperForConditionalGeneration.from_pretrained(model_name)


Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

In [51]:
!pip install -q huggingface_hub datasets


In [52]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("automatic-speech-recognition", model="openai/whisper-small")

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

In [53]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq

processor = AutoProcessor.from_pretrained("openai/whisper-small")
model = AutoModelForSpeechSeq2Seq.from_pretrained("openai/whisper-small")

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

In [54]:
#Converting raw data into whisper training format
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("openai/whisper-small")

def prepare_dataset(batch):
    audio = batch["audio"]

    batch["input_features"] = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"]
    ).input_features[0]

    batch["labels"] = processor.tokenizer(
        batch["text"],
        max_length=448,
        truncation=True
    ).input_ids


    return batch

dataset = dataset.map(prepare_dataset)
dataset=dataset.remove_columns(["audio", "text"])


Map:   0%|          | 0/104 [00:00<?, ? examples/s]

In [55]:
#creating a pytorch dataloader
from transformers import TrainingArguments
training_args=TrainingArguments(
    output_dir="./whisper-finetuned",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=1e-5,
    max_steps=500,
    logging_steps=10,
    save_steps=100,
    fp16=True
)

In [56]:
import torch
from torch.utils.data import DataLoader

dataset.set_format(type="torch")

train_loader = DataLoader(dataset, batch_size=4, shuffle=True)


In [57]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)


WhisperForConditionalGeneration(
  (model): WhisperModel(
    (encoder): WhisperEncoder(
      (conv1): Conv1d(80, 768, kernel_size=(3,), stride=(1,), padding=(1,))
      (conv2): Conv1d(768, 768, kernel_size=(3,), stride=(2,), padding=(1,))
      (embed_positions): Embedding(1500, 768)
      (layers): ModuleList(
        (0-11): 12 x WhisperEncoderLayer(
          (self_attn): WhisperAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=False)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (f

In [58]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)


In [59]:
def collate_fn(batch):
    input_features = torch.stack([x["input_features"] for x in batch])

    labels = [x["labels"] for x in batch]

    labels = processor.tokenizer.pad(
        {"input_ids": labels},
        return_tensors="pt"
    ).input_ids

    labels[labels == processor.tokenizer.pad_token_id] = -100

    return {
        "input_features": input_features,
        "labels": labels
    }

train_loader = DataLoader(dataset, batch_size=4, shuffle=True, collate_fn=collate_fn)


In [60]:
model.train()

for epoch in range(3):
    print(f"Epoch {epoch+1}")
    total_loss = 0

    for batch in tqdm(train_loader):
        input_features = batch["input_features"].to(device)
        labels = batch["labels"].to(device)

        outputs = model(input_features=input_features, labels=labels)
        loss = outputs.loss

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print("Average Loss:", total_loss / len(train_loader))


Epoch 1


100%|██████████| 26/26 [29:50<00:00, 68.88s/it]


Average Loss: 1.527741482624641
Epoch 2


100%|██████████| 26/26 [27:43<00:00, 63.98s/it]


Average Loss: 1.227468660244575
Epoch 3


100%|██████████| 26/26 [27:37<00:00, 63.76s/it]

Average Loss: 1.081710207920808


In [61]:
save_path = "/kaggle/working/whisper_finetuned"

os.makedirs(save_path, exist_ok=True)

model.save_pretrained(save_path)
processor.save_pretrained(save_path)

print("Model saved successfully at:", save_path)

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Model saved successfully at: /kaggle/working/whisper_finetuned


In [63]:
from jiwer import wer
from tqdm import tqdm
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
device="cpu"

model=AutoModelForSpeechSeq2Seq.from_pretrained("/kaggle/working/whisper_finetuned")
processor=AutoProcessor.from_pretrained("/kaggle/working/whisper_finetuned")

subset=dataset.select(range(20))

pred=[]
ref=[]

for sample in tqdm(subset):
    input_features=sample["input_features"].unsqueeze(0)
    with torch.no_grad():
        predicted_ids=model.generate(input_features)
    pred_text=processor.decode(predicted_ids[0], skip_special_tokens=True)
    ref_text=processor.decode(sample["labels"], skip_special_tokens=True)
    pred.append(pred_text)
    ref.append(ref_text)
wer_score=wer(ref, pred)
print("WER of dataset: ",wer_score)

Loading weights:   0%|          | 0/479 [00:00<?, ?it/s]

100%|██████████| 20/20 [16:02<00:00, 48.15s/it]

WER of dataset:  0.93419740777667


# Question 2

In [2]:
import os

os.listdir("/kaggle/working")


['.virtual_documents']